In [543]:
import snappy
import ast
import csv
import pickle
import random
import time 
from ast import literal_eval
from itertools import product

Let's import simplified diagrams of the census knots, which have not be determined to be strongly quasipositive () for those the Seifert genus already gives us the slice genus) 

In [544]:
######### braid words
braid_words_file=open('/Users/leomousseau/Documents/census-seminar/seminar-2021/braid_words.txt','r')
braid_words_string=braid_words_file.read()
braid_words_list=ast.literal_eval(braid_words_string)
braid_dict=dict(braid_words_list)

# 3 genus
genus=[]
with open('/Users/leomousseau/Documents/census-seminar/seminar-2021/3_genus,top_smooth_4_genus,unkotting_number,double_slice_genus/3genus.csv','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        genus.append(row)
del genus[0]
for i in range(len(genus)):
    genus[i][1]=int(genus[i][1])
genus_dict=dict(genus)

pd_codes_not_SQP=[]
with open('/Users/leomousseau/Desktop/Bachelor code census/pd_codes_not_SQP','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        pd_codes_not_SQP.append(row)
for k in pd_codes_not_SQP:
    k[1]=literal_eval(k[1])

In [545]:
len(pd_codes_not_SQP)

497

In [546]:
# lets remove the low crossing knots, since they are all prime (since they are hyperbolic) and therefore
# their slice genera can be obtained from KnotInfo
pd_codes=[]
for k in pd_codes_not_SQP:
    if len(k[1])<13:
        None
    else:
        pd_codes.append(k)


In [547]:
len(pd_codes)

409

In [548]:
def cc(pd,i):
    '''
    Changes the i-th crossing.
    '''
    PD_code=pd.copy()
    (a,b,c,d)=PD_code[i]
    PD_code[i]=(d,a,b,c)
    return PD_code

def pos_crossing(pd,i):
    '''
    True if the i-th crossing is positive, False else
    '''
    n=max([max(k) for k in pd])
    (a,b,c,d)=pd[i]
    if d==n:
        if b==0:
            return(True)
    if b==n:
        if d==0:
            return(False)
    if d<b:
        return(True)
    return(False)



In [549]:
def crossing_changer(pd):
    '''
    input: A pd-code
    output: list of pd codes corresponding crossing changes in pd
    '''
    l=[]
    for i in range(len(pd)):
        new_pd=cc(pd,i)
        l.append([new_pd,i,'cc'])
    return(l)


def two_crossing_changer(pd):
    '''
    input: pd code
    output: returns list of pd codes corresponding to changement of a positive and a negative crossing
    '''
    l=[]
    for i in range(len(pd)):
        if pos_crossing(pd,i):
            for j in range(len(pd)):
                if not pos_crossing(pd,j) and i!=j:
                    l.append([cc(cc(pd,i),j),(i,j),'cc+-'])
    return(l)

In [ ]:
def resolution(pd2,i):
    '''
    resolutes the i-th crossing
    '''
    pd=pd2.copy()
    pd=[list(k) for k in pd]
    (a,b,c,d)=pd[i]
    new_pd=[]
    if pos_crossing(pd,i):
        for j in range(len(pd)):
            p=[]
            t=pd[j]
            for e in t:
                if e==d:
                    e=c
                if e==b:
                    e=a
                p.append(e)
            new_pd.append(p)
    else:
        for j in range(len(pd)):
            p=[]
            t=pd[j]
            for e in t:
                if e==d:
                    e=a
                if e==b:
                    e=c
                p.append(e)
            new_pd.append(p)  
    del new_pd[i]
    return(new_pd)

In [ ]:
def resoluter(PD):
    '''
    input: A PD-code
    output: a list of PD codes all obtained from doing two oriented resolutions
    '''
    l=[]
    PDD=PD.copy()
    # first validate the pd code
    VALD = []  # validate the PD code!
    for ss in range(0, len(PDD)):
        if PDD[ss][2] != PDD[ss][0] + 1:
            if PDD[ss][0] == 2 * len(PDD) - 1:
                if PDD[ss][2] == 0:
                    VALD = VALD + [1]
                else:
                    VALD = VALD + [0]
            else:
                VALD = VALD + [0]
        else:
            VALD = VALD + [1]
    passit = 1
    for tt in range(0, len(VALD)):
        if VALD[tt] == 0:
            passit = 0
    #print(passit)
    if passit == 0:  # PD code is "invalid": need PDD[ss][2]=PDD[ss][0]+1 (except for possible wraparound)
        None #fb.write(str(PDD) + '\n')
    else:
        for hh in range(0, len(PDD)):
            PDDB = PDD[:]
            PRS = find_pairs(PDDB)
            good1 = test_crossing(hh, PDDB, PRS)
            if good1 == 1:
                PDNEW1 = resolve_crossing(hh, PDDB, PRS)
                PRS1 = find_pairs(PDNEW1)
                if len(PRS1) == 2:
                    # small fix: the find_pairs code misses a little loop around a single strand....
                    for kk in range(0,len(PDNEW1)): #max(0, hh - 1), len(PDNEW1)):
                        #print(hh,kk)
                        good2 = test_crossing(kk, PDNEW1, PRS1)
                        if good2 == 1:
                            PDNEW2 = resolve_crossing(kk, PDNEW1, PRS1)
                            try:
                                LX = snappy.Link(PDNEW2)
                                l.append([PDNEW2,(hh,kk),'res'])
                            except:
                                None
                                #print('pd code inconsistent')
            
    return(l)


In [ ]:
def find_pairs(PDC):    ## this finds the components in a PD code, as pairs of integers
    PRS=[]                               ## note they are *not* in increasing order...
    for ii in range(0,len(PDC)):         ## although each pair *is*
        if PDC[ii][2] != PDC[ii][0]+1:
            if PDC[ii][2] != PDC[ii][0]-1: 
                if PDC[ii][0] > PDC[ii][2]:
                    PRS=PRS+[[PDC[ii][2],PDC[ii][0]]]
                else:
                    PRS=PRS+[[PDC[ii][0],PDC[ii][2]]]
        if PDC[ii][3] != PDC[ii][1]+1:
            if PDC[ii][3] != PDC[ii][1]-1: 
                if PDC[ii][1] > PDC[ii][3]:
                    PRS=PRS+[[PDC[ii][3],PDC[ii][1]]]
                else:
                    PRS=PRS+[[PDC[ii][1],PDC[ii][3]]]
    return(PRS)

def test_crossing(nn,PDC,PRS):    ## test if the crossing, when resolved, 
    if len(PRS) > 2:                ## is 1-to-2 or 2-to-1
        good=0
    if len(PRS)==1:
        good=1
    if len(PRS)==2:    ## check if the crossing resolution will create a knot
        CROS=PDC[nn]     ## essentially, we need the two components to meet here!
        good=0
        if CROS[0] in range(PRS[0][0],PRS[0][1]+1):
            if CROS[1] in range(PRS[1][0],PRS[1][1]+1):
                good=1
        else:
            if CROS[0] in range(PRS[1][0],PRS[1][1]+1):
                if CROS[1] in range(PRS[0][0],PRS[0][1]+1):
                    good=1
    return(good)

## Stick with knots! So for our purposes, 
## only need to worry about two cases:
## PRS=[[0,2n-1]] and PRS=[[0,x],[x+1,2n-1]]
## Better: make sure in the second case that it will
## result in a knot!

In [1]:
def resolve_crossing(nn, PDC, PRS):
    PDNEW = []
    if len(PRS) == 1:
        CROS = PDC[nn]
        if CROS[2] == 0:
            ctu = min(PDC[nn][1], PDC[nn][3])
            for ee in range(0, len(PDC)):
                if ee != nn:
                    ADME = []
                    for aa in range(0, 4):
                        if PDC[ee][aa] in range(0, ctu):
                            ADME = ADME + [PDC[ee][aa]]
                        if PDC[ee][aa] == ctu:
                            ADME = ADME + [0]
                        if PDC[ee][aa] in range(ctu + 1, 2 * len(PDC) - 1):
                            ADME = ADME + [PDC[ee][aa] - 1]
                        if PDC[ee][aa] == 2 * len(PDC) - 1:
                            ADME = ADME + [ctu]
                    PDNEW = PDNEW + [ADME]
        else:
            if min(CROS[1], CROS[3]) == 0:
                for ee in range(0, len(PDC)):
                    if ee != nn:
                        ADME = []
                        for aa in range(0, 4):
                            if PDC[ee][aa] in range(0, CROS[0]):
                                ADME = ADME + [PDC[ee][aa]]
                            if PDC[ee][aa] == CROS[0]:
                                ADME = ADME + [0]
                            if PDC[ee][aa] in range(CROS[0] + 1, 2 * len(PDC) - 1):
                                ADME = ADME + [PDC[ee][aa] - 1]
                            if PDC[ee][aa] == 2 * len(PDC) - 1:
                                ADME = ADME + [CROS[0]]
                        PDNEW = PDNEW + [ADME]
            else:
                mun = min(CROS)
                mux = max(CROS)
                for ee in range(0, len(PDC)):
                    if ee != nn:
                        ADME = []
                        for aa in range(0, 4):
                            if PDC[ee][aa] in range(0, mun + 1):
                                ADME = ADME + [PDC[ee][aa]]
                            if PDC[ee][aa] in range(mux + 1, 2 * len(PDC)):
                                ADME = ADME + [PDC[ee][aa] + mun - mux]
                            if PDC[ee][aa] == mux:
                                ADME = ADME + [mun]
                            if PDC[ee][aa] == mux - 1:
                                ADME = ADME + [2 * len(PDC) + mun - mux]
                            if PDC[ee][aa] in range(mun + 1, mux - 1):
                                ADME = ADME + [PDC[ee][aa] + 2 * len(PDC) - mux - 1]
                        PDNEW = PDNEW + [ADME]
            
    if len(PRS) == 2:
        ## if we have run test_crossing, we know that the two components meet
        ENDSLO = []
        ENDSHI = []
        for ii in range(0, len(PRS)):
            ENDSLO = ENDSLO + [PRS[ii][0]]  ## smallest index on each component
            ENDSHI = ENDSHI + [PRS[ii][1]]  ## largest index on each component
        munl = min(ENDSLO)
        munh = max(ENDSLO)
        muxl = min(ENDSHI)
        muxh = max(ENDSHI)
        CROS = PDC[nn]
        if CROS[0] < CROS[1]:  ## the undercrossing component is labeled lower
            if CROS[1] == munh:
                if CROS[3] == munh + 1:
                    TPP = 'RL'
                else:
                    TPP = 'LR'  ## right side init, left to right
            else:
                if CROS[3] == munh:
                    if CROS[1] == munh + 1:
                        TPP = 'LR'
                    else:  ## left side init, right to left
                        TPP = 'RL'
                else:
                    if CROS[1] < CROS[3]:  ## no init, right to left
                        TPP = 'RL'
                    else:  ## no init, left to right
                        TPP = 'LR'
            if TPP == 'LR':  ## undercrossing lower, left to right
                for ee in range(0, len(PDC)):
                    if ee != nn:
                        ADME = []
                        if CROS[2] == 0:
                            if CROS[1] == munh:  ## edge case: both start lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[0] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[1], 2 * len(PDC) - 1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] == 2 * len(PDC) - 1:
                                        ADME = ADME + [0]
                            else:  ## lower 0, upper not lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[0] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[1], 2 * len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] in range(munh, CROS[3]):
                                        ADME = ADME + [PDC[ee][aa] - munh + 2 * len(PDC) - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] == CROS[3]:
                                        ADME = ADME + [0]
                     
                        else:
                            if CROS[1] == munh:  ## lower not lowest, upper is
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[0]+1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[1], 2*len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] in range(CROS[2], muxl+1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + 2*len(PDC) - 1 - CROS[1] + CROS[0]]
                            else:  ## neither is lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[0]+1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[1], 2*len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] in range(munh, CROS[3]+1):
                                        ADME = ADME + [PDC[ee][aa] - munh + 2*len(PDC) - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] in range(CROS[2], muxl+1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3] - munh + 2*len(PDC) - CROS[1] + CROS[0]]
                            PDNEW = PDNEW + [ADME]

                                                
                    else:  ## undercrossing lower, right to left
                        for ee in range(0,len(PDC)):
                            if ee != nn:
                                ADME=[]
                                if CROS[2]==0:
                                    if CROS[3]==munh:  ## edge case: both start lowest
                                        for aa in range(0,4):
                                            if PDC[ee][aa] in range(0,CROS[0]+1):
                                                ADME=ADME+[PDC[ee][aa]]
                                            if PDC[ee][aa] in range(CROS[3],2*len(PDC)-1):
                                                ADME=ADME+[PDC[ee][aa]-CROS[3]+CROS[0]]
                                            if PDC[ee][aa]==2*len(PDC)-1:
                                                ADME=ADME+[0]
                                    else:  ## lower 0, upper not lowest
                                        for aa in range(0,4):
                                            if PDC[ee][aa] in range(0,CROS[0]+1):
                                                ADME=ADME+[PDC[ee][aa]]
                                            if PDC[ee][aa] in range(CROS[3],2*len(PDC)):
                                                ADME=ADME+[PDC[ee][aa]-CROS[3]+CROS[0]]
                                            if PDC[ee][aa] in range(munh,CROS[1]):
                                                ADME=ADME+[PDC[ee][aa]-munh+2*len(PDC)-CROS[3]+CROS[0]]
                                            if PDC[ee][aa]==CROS[1]:
                                                ADME=ADME+[0]
                                else:
                                    if CROS[3]==munh:  ## lower not lowest, upper is
                                        for aa in range(0,4):
                                            if PDC[ee][aa] in range(0,CROS[0]+1):
                                                ADME=ADME+[PDC[ee][aa]]
                                            if PDC[ee][aa] in range(CROS[3],2*len(PDC)):
                                                ADME=ADME+[PDC[ee][aa]-CROS[3]+CROS[0]]
                                            if PDC[ee][aa] in range(CROS[2],muxl+1):
                                                ADME=ADME+[PDC[ee][aa]-CROS[2]+2*len(PDC)-1-CROS[3]+CROS[0]]
                                    
                                                                                                  
                                    else:  ## neither is lowest
                                        for aa in range(0, 4):
                                            if PDC[ee][aa] in range(0, CROS[0] + 1):
                                                ADME = ADME + [PDC[ee][aa]]
                                            if PDC[ee][aa] in range(CROS[3], 2 * len(PDC)):
                                                ADME = ADME + [PDC[ee][aa] - CROS[3] + CROS[0]]
                                            if PDC[ee][aa] in range(munh, CROS[1] + 1):
                                                ADME = ADME + [PDC[ee][aa] - munh + 2 * len(PDC) - CROS[3] + CROS[0]]
                                            if PDC[ee][aa] in range(CROS[2], muxl + 1):
                                                ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[1] - munh + 2 * len(PDC) - CROS[3] + CROS[0]]
                                PDNEW = PDNEW + [ADME]
                                                                                                  
        else:  ## the overcrossing component is labeled lower
            if CROS[1] == munl:                   
                if CROS[3] == munl + 1:
                    TPP = 'RL'
                else:  ## right side init, left to right
                    TPP = 'LR'
            else:
                if CROS[3] == munl:                 
                    if CROS[1] == munl + 1:
                        TPP = 'LR'
                    else:
                        TPP = 'RL'  ## left side init, right to left
                else:
                    if CROS[1] < CROS[3]:  ## no init, right to left
                        TPP = 'RL'             
                    else:  ## no init, left to right
                        TPP = 'LR'
            if TPP == 'LR':  ##  overcrossing lower, left to right
                for ee in range(0, len(PDC)):
                    if ee != nn:
                        ADME = []
                        if CROS[2] == munh:
                            if CROS[1] == 0:  ## edge case: both start lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[3] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2], 2 * len(PDC) - 1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] == 2 * len(PDC) - 1:
                                        ADME = ADME + [0]
                            else:  ## lower munh, upper not lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[3] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2], 2 * len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] in range(CROS[1], muxl + 1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + 2 * len(PDC) - 1 - CROS[2] + CROS[3]]
                       
                        else:
                            if CROS[1] == 0:  ## lower not lowest, upper is
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[3] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2], 2 * len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] in range(munh, CROS[0]):
                                        ADME = ADME + [PDC[ee][aa] - munh + 2 * len(PDC) - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] == CROS[0]:
                                        ADME = ADME + [0]
                            else:  ## neither is lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[3] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2], 2 * len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] in range(munh, CROS[0] + 1):
                                        ADME = ADME + [PDC[ee][aa] - munh + 2 * len(PDC) - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] in range(CROS[1], muxl + 1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0] - munh + 2 * len(PDC) - CROS[2] + CROS[3]]
                        PDNEW = PDNEW + [ADME]
                                                                                                  
            

            else:
                ## overcrossing lower, right to left
                for ee in range(0,len(PDC)):
                    if ee != nn:
                        ADME=[]
                        if CROS[2]==munh:
                            if CROS[3]==0:          ## edge case: both start lowest
                                for aa in range(0,4):
                                    if PDC[ee][aa] in range(0,CROS[1]+1):
                                        ADME=ADME+[PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2],2*len(PDC)-1):
                                        ADME=ADME+[PDC[ee][aa]-CROS[2]+CROS[1]]
                                    if PDC[ee][aa]==2*len(PDC)-1:
                                        ADME=ADME+[0]
                            else:                     ## lower munh, upper not lowest
                                for aa in range(0,4):
                                    if PDC[ee][aa] in range(0,CROS[1]+1):
                                        ADME=ADME+[PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2],2*len(PDC)):
                                        ADME=ADME+[PDC[ee][aa]-CROS[2]+CROS[1]]
                                    if PDC[ee][aa] in range(CROS[3],muxl+1):
                                        ADME=ADME+[PDC[ee][aa]-CROS[3]+2*len(PDC)-1-CROS[2]+CROS[1]]
                        else:
                            if CROS[3]==0:        ## lower not lowest, upper is
                                for aa in range(0,4):
                                    if PDC[ee][aa] in range(0,CROS[1]+1):
                                        ADME=ADME+[PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2],2*len(PDC)):
                                        ADME=ADME+[PDC[ee][aa]-CROS[2]+CROS[1]]
                                    if PDC[ee][aa] in range(munh,CROS[0]):
                                        ADME=ADME+[PDC[ee][aa]-munh+2*len(PDC)-CROS[2]+CROS[1]]
                                    if PDC[ee][aa]==CROS[0]:
                                        ADME=ADME+[0]                  
                            else:                    ## neither is lowest
                                for aa in range(0,4):
                                    if PDC[ee][aa] in range(0,CROS[1]+1):
                                        ADME=ADME+[PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2],2*len(PDC)):
                                        ADME=ADME+[PDC[ee][aa]-CROS[2]+CROS[1]]
                                    if PDC[ee][aa] in range(munh,CROS[0]+1):
                                        ADME=ADME+[PDC[ee][aa]-munh+2*len(PDC)-CROS[2]+CROS[1]]
                                    if PDC[ee][aa] in range(CROS[3],muxl+1):
                                        ADME=ADME+[PDC[ee][aa]-CROS[3]+CROS[0]-munh+2*len(PDC)-CROS[2]+CROS[1]]
                        PDNEW=PDNEW+[ADME]
    return(PDNEW)

In [550]:
def test_crossing(nn,PDC,PRS):    ## test if the crossing, when resolved, 
    if len(PRS) > 2:                ## is 1-to-2 or 2-to-1
        good=0
    if len(PRS)==1:
        good=1
    if len(PRS)==2:    ## check if the crossing resolution will create a knot
        CROS=PDC[nn]     ## essentially, we need the two components to meet here!
        good=0
        if CROS[0] in range(PRS[0][0],PRS[0][1]+1):
            if CROS[1] in range(PRS[1][0],PRS[1][1]+1):
                good=1
        else:
            if CROS[0] in range(PRS[1][0],PRS[1][1]+1):
                if CROS[1] in range(PRS[0][0],PRS[0][1]+1):
                    good=1
    return(good)

## Stick with knots! So for our purposes, 
## only need to worry about two cases:
## PRS=[[0,2n-1]] and PRS=[[0,x],[x+1,2n-1]]
## Better: make sure in the second case that it will
## result in a knot!

In [551]:
def find_pairs(PDC):    ## this finds the components in a PD code, as pairs of integers
    PRS=[]                               ## note they are *not* in increasing order...
    for ii in range(0,len(PDC)):         ## although each pair *is*
        if PDC[ii][2] != PDC[ii][0]+1:
            if PDC[ii][2] != PDC[ii][0]-1: 
                if PDC[ii][0] > PDC[ii][2]:
                    PRS=PRS+[[PDC[ii][2],PDC[ii][0]]]
                else:
                    PRS=PRS+[[PDC[ii][0],PDC[ii][2]]]
        if PDC[ii][3] != PDC[ii][1]+1:
            if PDC[ii][3] != PDC[ii][1]-1: 
                if PDC[ii][1] > PDC[ii][3]:
                    PRS=PRS+[[PDC[ii][3],PDC[ii][1]]]
                else:
                    PRS=PRS+[[PDC[ii][1],PDC[ii][3]]]
    return(PRS)

In [691]:
def resoluter(PD):
    '''
    input: A PD-code
    output: a list of PD codes all obtained from doing two oriented resolutions
    '''
    l=[]
    PDD=PD.copy()
    # first validate the pd code
    VALD = []  # validate the PD code!
    for ss in range(0, len(PDD)):
        if PDD[ss][2] != PDD[ss][0] + 1:
            if PDD[ss][0] == 2 * len(PDD) - 1:
                if PDD[ss][2] == 0:
                    VALD = VALD + [1]
                else:
                    VALD = VALD + [0]
            else:
                VALD = VALD + [0]
        else:
            VALD = VALD + [1]
    passit = 1
    for tt in range(0, len(VALD)):
        if VALD[tt] == 0:
            passit = 0
    #print(passit)
    if passit == 0:  # PD code is "invalid": need PDD[ss][2]=PDD[ss][0]+1 (except for possible wraparound)
        None #fb.write(str(PDD) + '\n')
    else:
        for hh in range(0, len(PDD)):
            PDDB = PDD[:]
            PRS = find_pairs(PDDB)
            good1 = test_crossing(hh, PDDB, PRS)
            if good1 == 1:
                PDNEW1 = resolve_crossing(hh, PDDB, PRS)
                PRS1 = find_pairs(PDNEW1)
                if len(PRS1) == 2:
                    # small fix: the find_pairs code misses a little loop around a single strand....
                    for kk in range(0,len(PDNEW1)): #max(0, hh - 1), len(PDNEW1)):
                        #print(hh,kk)
                        good2 = test_crossing(kk, PDNEW1, PRS1)
                        if good2 == 1:
                            PDNEW2 = resolve_crossing(kk, PDNEW1, PRS1)
                            try:
                                LX = snappy.Link(PDNEW2)
                                l.append([PDNEW2,(hh,kk),'res'])
                            except:
                                None
                                #print('pd code inconsistent')
            
    return(l)


In [711]:
def func(pd4,counter=0,change_sequence=[]):
    '''
    input: A PD-code
    output: pd code, with list of operations done to obtain it
    '''
    pd=pd4.copy()
    if len(pd)<13:
        return([pd,counter,change_sequence])
    counter+=1
    new_pds1=two_crossing_changer(pd)+crossing_changer(pd)#+resoluter(pd)
    new_pds2=[]
    for k in range(len(new_pds1)):
        t=new_pds1[k]
        pd_new=t[0]
        K=snappy.Link(pd_new)
        K.simplify('basic')
        new_pds2.append([K.PD_code(),len(K.crossings),k])
        
    n=min([j[1] for j in new_pds2])
    best_ones=[] 
    for j in new_pds2:
        if j[1]==n:
            best_ones.append([j[0],j[2]])
            
    [best_pd,index]=[best_ones[0][0],best_ones[0][1]]
    [place,operation]=[new_pds1[index][1],new_pds1[index][2]]
    change_sequence.append([pd,(place,operation)])
    print(counter)
    return(func(best_pd,counter,change_sequence))

In [712]:
pd=pd_codes[50][1]
K=snappy.Link(pd)
K.knot_floer_homology()['tau']

5

In [714]:
func(pd,0,[])[:2]

1
2
3
4
5


[[(7, 1, 8, 0),
  (2, 10, 3, 9),
  (6, 14, 7, 13),
  (19, 15, 20, 14),
  (5, 19, 6, 18),
  (21, 9, 0, 8),
  (11, 5, 12, 4),
  (16, 4, 17, 3),
  (15, 11, 16, 10),
  (17, 12, 18, 13),
  (20, 2, 21, 1)],
 5]

In [696]:
l=[]
i=0
for [name,pd] in pd_codes[:400]:
    l.append([name,func(pd,0,[])])
    #L=snappy.Link(pd)
    #tau=L.knot_floer_homology()['tau']
    #print(round(time.time()-start),i)
    #start=time.time()


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [695]:
l[8]

['v0595',
 [[],
  1,
  [[[(34, 28, 35, 27),
     (35, 17, 0, 16),
     (17, 33, 18, 32),
     (28, 34, 29, 33),
     (6, 30, 7, 29),
     (31, 9, 32, 8),
     (30, 20, 31, 19),
     (18, 7, 19, 8),
     (9, 5, 10, 4),
     (20, 6, 21, 5),
     (10, 21, 11, 22),
     (2, 12, 3, 11),
     (3, 23, 4, 22),
     (23, 12, 24, 13),
     (13, 24, 14, 25),
     (26, 15, 27, 16),
     (14, 2, 15, 1),
     (25, 1, 26, 0)],
    ((0, 4), 'res')]]]]

In [579]:
def resolve_crossing(nn, PDC, PRS):
    PDNEW = []
    if len(PRS) == 1:
        CROS = PDC[nn]
        if CROS[2] == 0:
            ctu = min(PDC[nn][1], PDC[nn][3])
            for ee in range(0, len(PDC)):
                if ee != nn:
                    ADME = []
                    for aa in range(0, 4):
                        if PDC[ee][aa] in range(0, ctu):
                            ADME = ADME + [PDC[ee][aa]]
                        if PDC[ee][aa] == ctu:
                            ADME = ADME + [0]
                        if PDC[ee][aa] in range(ctu + 1, 2 * len(PDC) - 1):
                            ADME = ADME + [PDC[ee][aa] - 1]
                        if PDC[ee][aa] == 2 * len(PDC) - 1:
                            ADME = ADME + [ctu]
                    PDNEW = PDNEW + [ADME]
        else:
            if min(CROS[1], CROS[3]) == 0:
                for ee in range(0, len(PDC)):
                    if ee != nn:
                        ADME = []
                        for aa in range(0, 4):
                            if PDC[ee][aa] in range(0, CROS[0]):
                                ADME = ADME + [PDC[ee][aa]]
                            if PDC[ee][aa] == CROS[0]:
                                ADME = ADME + [0]
                            if PDC[ee][aa] in range(CROS[0] + 1, 2 * len(PDC) - 1):
                                ADME = ADME + [PDC[ee][aa] - 1]
                            if PDC[ee][aa] == 2 * len(PDC) - 1:
                                ADME = ADME + [CROS[0]]
                        PDNEW = PDNEW + [ADME]
            else:
                mun = min(CROS)
                mux = max(CROS)
                for ee in range(0, len(PDC)):
                    if ee != nn:
                        ADME = []
                        for aa in range(0, 4):
                            if PDC[ee][aa] in range(0, mun + 1):
                                ADME = ADME + [PDC[ee][aa]]
                            if PDC[ee][aa] in range(mux + 1, 2 * len(PDC)):
                                ADME = ADME + [PDC[ee][aa] + mun - mux]
                            if PDC[ee][aa] == mux:
                                ADME = ADME + [mun]
                            if PDC[ee][aa] == mux - 1:
                                ADME = ADME + [2 * len(PDC) + mun - mux]
                            if PDC[ee][aa] in range(mun + 1, mux - 1):
                                ADME = ADME + [PDC[ee][aa] + 2 * len(PDC) - mux - 1]
                        PDNEW = PDNEW + [ADME]
            
    if len(PRS) == 2:
        ## if we have run test_crossing, we know that the two components meet
        ENDSLO = []
        ENDSHI = []
        for ii in range(0, len(PRS)):
            ENDSLO = ENDSLO + [PRS[ii][0]]  ## smallest index on each component
            ENDSHI = ENDSHI + [PRS[ii][1]]  ## largest index on each component
        munl = min(ENDSLO)
        munh = max(ENDSLO)
        muxl = min(ENDSHI)
        muxh = max(ENDSHI)
        CROS = PDC[nn]
        if CROS[0] < CROS[1]:  ## the undercrossing component is labeled lower
            if CROS[1] == munh:
                if CROS[3] == munh + 1:
                    TPP = 'RL'
                else:
                    TPP = 'LR'  ## right side init, left to right
            else:
                if CROS[3] == munh:
                    if CROS[1] == munh + 1:
                        TPP = 'LR'
                    else:  ## left side init, right to left
                        TPP = 'RL'
                else:
                    if CROS[1] < CROS[3]:  ## no init, right to left
                        TPP = 'RL'
                    else:  ## no init, left to right
                        TPP = 'LR'
            if TPP == 'LR':  ## undercrossing lower, left to right
                for ee in range(0, len(PDC)):
                    if ee != nn:
                        ADME = []
                        if CROS[2] == 0:
                            if CROS[1] == munh:  ## edge case: both start lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[0] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[1], 2 * len(PDC) - 1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] == 2 * len(PDC) - 1:
                                        ADME = ADME + [0]
                            else:  ## lower 0, upper not lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[0] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[1], 2 * len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] in range(munh, CROS[3]):
                                        ADME = ADME + [PDC[ee][aa] - munh + 2 * len(PDC) - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] == CROS[3]:
                                        ADME = ADME + [0]
                     
                        else:
                            if CROS[1] == munh:  ## lower not lowest, upper is
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[0]+1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[1], 2*len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] in range(CROS[2], muxl+1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + 2*len(PDC) - 1 - CROS[1] + CROS[0]]
                            else:  ## neither is lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[0]+1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[1], 2*len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] in range(munh, CROS[3]+1):
                                        ADME = ADME + [PDC[ee][aa] - munh + 2*len(PDC) - CROS[1] + CROS[0]]
                                    if PDC[ee][aa] in range(CROS[2], muxl+1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3] - munh + 2*len(PDC) - CROS[1] + CROS[0]]
                            PDNEW = PDNEW + [ADME]

                                                
                    else:  ## undercrossing lower, right to left
                        for ee in range(0,len(PDC)):
                            if ee != nn:
                                ADME=[]
                                if CROS[2]==0:
                                    if CROS[3]==munh:  ## edge case: both start lowest
                                        for aa in range(0,4):
                                            if PDC[ee][aa] in range(0,CROS[0]+1):
                                                ADME=ADME+[PDC[ee][aa]]
                                            if PDC[ee][aa] in range(CROS[3],2*len(PDC)-1):
                                                ADME=ADME+[PDC[ee][aa]-CROS[3]+CROS[0]]
                                            if PDC[ee][aa]==2*len(PDC)-1:
                                                ADME=ADME+[0]
                                    else:  ## lower 0, upper not lowest
                                        for aa in range(0,4):
                                            if PDC[ee][aa] in range(0,CROS[0]+1):
                                                ADME=ADME+[PDC[ee][aa]]
                                            if PDC[ee][aa] in range(CROS[3],2*len(PDC)):
                                                ADME=ADME+[PDC[ee][aa]-CROS[3]+CROS[0]]
                                            if PDC[ee][aa] in range(munh,CROS[1]):
                                                ADME=ADME+[PDC[ee][aa]-munh+2*len(PDC)-CROS[3]+CROS[0]]
                                            if PDC[ee][aa]==CROS[1]:
                                                ADME=ADME+[0]
                                else:
                                    if CROS[3]==munh:  ## lower not lowest, upper is
                                        for aa in range(0,4):
                                            if PDC[ee][aa] in range(0,CROS[0]+1):
                                                ADME=ADME+[PDC[ee][aa]]
                                            if PDC[ee][aa] in range(CROS[3],2*len(PDC)):
                                                ADME=ADME+[PDC[ee][aa]-CROS[3]+CROS[0]]
                                            if PDC[ee][aa] in range(CROS[2],muxl+1):
                                                ADME=ADME+[PDC[ee][aa]-CROS[2]+2*len(PDC)-1-CROS[3]+CROS[0]]
                                    
                                                                                                  
                                    else:  ## neither is lowest
                                        for aa in range(0, 4):
                                            if PDC[ee][aa] in range(0, CROS[0] + 1):
                                                ADME = ADME + [PDC[ee][aa]]
                                            if PDC[ee][aa] in range(CROS[3], 2 * len(PDC)):
                                                ADME = ADME + [PDC[ee][aa] - CROS[3] + CROS[0]]
                                            if PDC[ee][aa] in range(munh, CROS[1] + 1):
                                                ADME = ADME + [PDC[ee][aa] - munh + 2 * len(PDC) - CROS[3] + CROS[0]]
                                            if PDC[ee][aa] in range(CROS[2], muxl + 1):
                                                ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[1] - munh + 2 * len(PDC) - CROS[3] + CROS[0]]
                                PDNEW = PDNEW + [ADME]
                                                                                                  
        else:  ## the overcrossing component is labeled lower
            if CROS[1] == munl:                   
                if CROS[3] == munl + 1:
                    TPP = 'RL'
                else:  ## right side init, left to right
                    TPP = 'LR'
            else:
                if CROS[3] == munl:                 
                    if CROS[1] == munl + 1:
                        TPP = 'LR'
                    else:
                        TPP = 'RL'  ## left side init, right to left
                else:
                    if CROS[1] < CROS[3]:  ## no init, right to left
                        TPP = 'RL'             
                    else:  ## no init, left to right
                        TPP = 'LR'
            if TPP == 'LR':  ##  overcrossing lower, left to right
                for ee in range(0, len(PDC)):
                    if ee != nn:
                        ADME = []
                        if CROS[2] == munh:
                            if CROS[1] == 0:  ## edge case: both start lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[3] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2], 2 * len(PDC) - 1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] == 2 * len(PDC) - 1:
                                        ADME = ADME + [0]
                            else:  ## lower munh, upper not lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[3] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2], 2 * len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] in range(CROS[1], muxl + 1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + 2 * len(PDC) - 1 - CROS[2] + CROS[3]]
                       
                        else:
                            if CROS[1] == 0:  ## lower not lowest, upper is
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[3] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2], 2 * len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] in range(munh, CROS[0]):
                                        ADME = ADME + [PDC[ee][aa] - munh + 2 * len(PDC) - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] == CROS[0]:
                                        ADME = ADME + [0]
                            else:  ## neither is lowest
                                for aa in range(0, 4):
                                    if PDC[ee][aa] in range(0, CROS[3] + 1):
                                        ADME = ADME + [PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2], 2 * len(PDC)):
                                        ADME = ADME + [PDC[ee][aa] - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] in range(munh, CROS[0] + 1):
                                        ADME = ADME + [PDC[ee][aa] - munh + 2 * len(PDC) - CROS[2] + CROS[3]]
                                    if PDC[ee][aa] in range(CROS[1], muxl + 1):
                                        ADME = ADME + [PDC[ee][aa] - CROS[1] + CROS[0] - munh + 2 * len(PDC) - CROS[2] + CROS[3]]
                        PDNEW = PDNEW + [ADME]
                                                                                                  
            

            else:
                ## overcrossing lower, right to left
                for ee in range(0,len(PDC)):
                    if ee != nn:
                        ADME=[]
                        if CROS[2]==munh:
                            if CROS[3]==0:          ## edge case: both start lowest
                                for aa in range(0,4):
                                    if PDC[ee][aa] in range(0,CROS[1]+1):
                                        ADME=ADME+[PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2],2*len(PDC)-1):
                                        ADME=ADME+[PDC[ee][aa]-CROS[2]+CROS[1]]
                                    if PDC[ee][aa]==2*len(PDC)-1:
                                        ADME=ADME+[0]
                            else:                     ## lower munh, upper not lowest
                                for aa in range(0,4):
                                    if PDC[ee][aa] in range(0,CROS[1]+1):
                                        ADME=ADME+[PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2],2*len(PDC)):
                                        ADME=ADME+[PDC[ee][aa]-CROS[2]+CROS[1]]
                                    if PDC[ee][aa] in range(CROS[3],muxl+1):
                                        ADME=ADME+[PDC[ee][aa]-CROS[3]+2*len(PDC)-1-CROS[2]+CROS[1]]
                        else:
                            if CROS[3]==0:        ## lower not lowest, upper is
                                for aa in range(0,4):
                                    if PDC[ee][aa] in range(0,CROS[1]+1):
                                        ADME=ADME+[PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2],2*len(PDC)):
                                        ADME=ADME+[PDC[ee][aa]-CROS[2]+CROS[1]]
                                    if PDC[ee][aa] in range(munh,CROS[0]):
                                        ADME=ADME+[PDC[ee][aa]-munh+2*len(PDC)-CROS[2]+CROS[1]]
                                    if PDC[ee][aa]==CROS[0]:
                                        ADME=ADME+[0]                  
                            else:                    ## neither is lowest
                                for aa in range(0,4):
                                    if PDC[ee][aa] in range(0,CROS[1]+1):
                                        ADME=ADME+[PDC[ee][aa]]
                                    if PDC[ee][aa] in range(CROS[2],2*len(PDC)):
                                        ADME=ADME+[PDC[ee][aa]-CROS[2]+CROS[1]]
                                    if PDC[ee][aa] in range(munh,CROS[0]+1):
                                        ADME=ADME+[PDC[ee][aa]-munh+2*len(PDC)-CROS[2]+CROS[1]]
                                    if PDC[ee][aa] in range(CROS[3],muxl+1):
                                        ADME=ADME+[PDC[ee][aa]-CROS[3]+CROS[0]-munh+2*len(PDC)-CROS[2]+CROS[1]]
                        PDNEW=PDNEW+[ADME]
    return(PDNEW)